### 📌 LangServe 개요
- LangChain Expression Language(LCEL)로 만든 체인/그래프/Agent 등을 **API 서버로 배포**하기 위한 도구
- 내부적으로 **FastAPI** + **Uvicorn** 기반
- 장점:
  - LangChain에서 만든 체인을 바로 서빙 가능
  - OpenAI API와 비슷한 호출 인터페이스 제공 (`/invoke`, `/stream`, `/batch`)
  - **Observability**: LangSmith와 연동하여 추적 가능
- 주요 사용처:
  - RAG, Agent, Chatbot 모델을 REST API로 제공
  - 사내 서비스/대시보드/앱에서 쉽게 활용


In [1]:
# # 설치
# # pip install "langserve[all]" fastapi uvicorn

## LangServe로 모델 서빙하기

# from fastapi import FastAPI
# from langserve import add_routes
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# from langchain.schema import StrOutputParser

# # 1) 체인 정의 (LangChain LCEL)
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 친절한 AI 비서입니다."),
#     ("human", "{question}")
# ])
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# chain = prompt | llm | StrOutputParser()

# # 2) FastAPI 앱 생성
# app = FastAPI(
#     title="LangServe Example",
#     version="1.0",
#     description="LangServe로 배포한 예시 체인"
# )

# # 3) 체인을 API 엔드포인트로 추가
# add_routes(app, chain, path="/chat")

# # 4) 실행 (터미널)
# # uvicorn filename:app --reload --port 8000

### 📌 서빙 구조
- `add_routes(app, chain, path="/chat")`
  - `/chat/invoke` : 단일 입력 → 출력
  - `/chat/stream` : 스트리밍 출력 (토큰 단위)
  - `/chat/batch`  : 여러 입력 동시에 처리
- 체인은 **LCEL**로 정의된 모든 객체 (프롬프트 → LLM → 파서)를 지원


In [ ]:
# # Python 클라이언트에서 호출
# from langserve.client import RemoteRunnable

# # LangServe 서버 주소 (로컬 또는 배포 서버)
# remote_chain = RemoteRunnable("http://localhost:8000/chat")

# # 단일 호출
# resp = remote_chain.invoke({"question": "LangServe가 뭐야?"})
# print("응답:", resp)

# # 배치 호출
# resps = remote_chain.batch([
#     {"question": "LangChain이 뭐야?"},
#     {"question": "LangGraph가 뭐야?"}
# ])
# print(resps)

# # 스트리밍 호출
# for chunk in remote_chain.stream({"question": "토큰 단위로 답변해줘"}):
#     print(chunk, end="")

In [ ]:
# # HTTP REST 호출
# # invoke (단일 입력)
# curl http://localhost:8000/chat/invoke \
#     -H "Content-Type: application/json" \
#     -d '{"input": {"question": "LangServe란?"}}'

# # batch (여러 입력)
# curl http://localhost:8000/chat/batch \
#     -H "Content-Type: application/json" \
#     -d '{"inputs": [{"question": "LangChain?"}, {"question": "LangGraph?"}]}'

# # stream (스트리밍)
# curl -N http://localhost:8000/chat/stream \
#     -H "Content-Type: application/json" \
#     -d '{"input": {"question": "스트리밍 예시"}}'

## 프론트엔드/다른 서비스 연동
- FastAPI 엔드포인트는 REST 규격이므로, **React, Vue, Next.js** 같은 프론트엔드에서 바로 호출 가능
- 회사 내부 백엔드 서비스 (Flask, Django, Node.js 등)와도 쉽게 통합
- LangServe는 LangChain Hub, LangSmith와 연동 가능 → 관리/모니터링 용이

## ✅ 최종 정리

- LangServe = LangChain 체인을 쉽게 REST API로 서빙하는 도구

- 서빙 절차:

    - 체인 정의 (프롬프트 → LLM → 파서)

    - add_routes(app, chain, path="...")

    - FastAPI + Uvicorn으로 실행

- 사용 방법:

    - Python 클라이언트(RemoteRunnable)

    - REST API (invoke, batch, stream)

- 활용 예시:

    - 사내 RAG 검색기 API

    - AI 챗봇 백엔드

    - 데이터 분석 Agent 서빙